In [1]:
import os
# os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/krishnaik06/datascienceproject.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="krishnaik06"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="7104284f1bb44ece21e0e2adb4e36a250ae3251f"

In [2]:
import os
%pwd

'f:\\MLOps\\DataScienceProjectWithDeployment\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\MLOps\\DataScienceProjectWithDeployment'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.RandomForestClassifier
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="http://127.0.0.1:5000"
        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)
        return precision, recall, roc_auc
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (precision, recall, roc_auc) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"Precision": precision, "Recall": recall, "ROC-AUC": roc_auc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("Precision", precision)
            mlflow.log_metric("Recall", recall)
            mlflow.log_metric("ROC-AUC", roc_auc)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-19 22:56:09,878: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-19 22:56:09,884: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-19 22:56:09,895: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-19 22:56:09,897: INFO: common: Created directory at: artifacts]
[2025-10-19 22:56:09,901: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-10-19 22:56:11,353: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/10/19 22:56:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 22:56:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
